# Notebook 1: MED-PC Extracting the Recording Data and Metadata

# Importing the Python Libraries

In [1]:
import sys
import glob
from collections import defaultdict
import os
from datetime import datetime

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from medpc2excel.medpc_read import medpc_read

In [3]:
# setting path
sys.path.append('../../src')

In [4]:
# All the libraries that were created for this repository
import extract.dataframe
import processing.tone
import extract.metadata

# Getting the Metadata from all the MED-PC Recording Files

- Every MED-PC Recording file has Metadata about time, data, subject, group, experiment name, script used, and the MED-PC box number on the top of the recording file. We will first extract that information
    - Metadata can be background information or context about data(or files) that is often different from the actual data that is intended to be recorded.  
        For more information on Metadata: https://en.wikipedia.org/wiki/Metadata

In [5]:
all_med_pc_file = glob.glob("./data/**/*.txt", recursive=True)

- The path of the directory that this notebook is in. This will be where relative paths will be based off of

In [6]:
current_working_directory = os.getcwd()

In [7]:
current_working_directory

- All the other files in this directory. If you want to use a folder in here such as the `data` folder, you'd type `./data/{name_of_folder}` where you replace `{name_of_folder}` with the name of the folder without the `{}`. You will do this in the cells following the one below
    - The `./` means the path will reference the current directory that the command is being used from

In [8]:
os.listdir(current_working_directory)

## NOTE: If you are using your own data, the following path must be changed to the directory where your MED-PC recording files are located(if they are not in the specied folder). It is recommended to create a sub folder in the `data` folder(that is in the same folder as this notebook) for each group of session you want to process and put the recording files in there

- Use the cell below instead if you're using your own data. You must change the cell below by clicking on the cell and then pressing the `esc` key. Then press the `Y` button on your keyboard. If you want to switch it back do the same but press the `R` button instead. This switches it from a cell that is run as if it's code, or back to a cell where it's just treated as text
    - **NOTE: If you are using a different folder, then change the path in the cell below**
    - The asterisk is called a wild card which will tell the computer to find all files/folders that match this pattern. The `*` means that any pattern of any length can be replaced with it. The `**` tells the computer to look in all folders that are in the specified folder. For more information: https://linuxhint.com/bash_wildcard_tutorial/
    - For more information on finding the path of your folder that contains the recording files: https://www.computerhope.com/issues/ch001708.htm

In [9]:
all_med_pc_file = glob.glob("./data/*.txt")

In [10]:
all_med_pc_file[:10]

- Example of what the MED-PC Recording file looks like

In [11]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

- We will be extracting the first 10 or so lines that look like:

```
File: C:\MED-PC\Data\2022-05-06_12h59m_Subject 3.4 (2).txt

Start Date: 05/06/22

End Date: 05/06/22

Subject: 3.4 (2)

Experiment: Pilot of Pilot

Group: Cage 4

Box: 1

Start Time: 12:59:58

End Time: 14:02:38

MSN: levelNP_CS_reward_laserepochON1st_noshock
```

- We will just find all the lines that start with `"File", "Start Date", "End Date", "Subject", "Experiment", "Group", "Box", "Start Time", "End Time", or "MSN"`. And get the metadata from those lines. And then stop once all the metadata types have been collected

In [12]:
# This makes a nested dictionary of file paths to each individual metadata type
file_path_to_meta_data = extract.metadata.get_all_med_pc_meta_data_from_files(list_of_files=all_med_pc_file)

In [13]:
# The metadata for the first file
for key, value in file_path_to_meta_data.items():
    print("File path: {}".format(key))
    print("Metadata types and associated values: {}".format(value))
    break

## Making a Dataframe out of the Metadata

- A Dataframe is essentially a "programmable" spreadsheet. But instead of clicking on cells, you will have to tell Python how you want to interact with the spreadsheet
    - For more information: https://realpython.com/pandas-dataframe/

In [14]:
# Turning the dictionary into a Pandas Dataframe
metadata_df = pd.DataFrame.from_dict(file_path_to_meta_data, orient="index")
# Resetting the index because currently the file path is the index 
metadata_df = metadata_df.reset_index()

In [15]:
metadata_df.head()

In [16]:
metadata_df.tail()

- Getting the numbers out of the column that contains the cage information

In [17]:
metadata_df["cage"] = metadata_df["Group"].apply(lambda x: x.lower().strip("cage").strip())

In [18]:
metadata_df["cage"].head()

- Getting the number of files that are associated with each subject

In [19]:
# How many files there are for each subject
metadata_df.groupby("Subject").count()

## Looking over the MED-PC scripts

- MED-PC has scripts(lines of code that act as the instructions for how to operate the MED-PC boxes) that contain the descriptions of what each value in the recordings are. We will extract the descriptions from the MED-PC scripts so that we can label the data points in the MED-PC recordings  
- Below is a list of all the MED-PC Scripts. We will only be looking at the first one    

In [20]:
all_medpc_scripts = glob.glob("./**/*.MPC")

In [21]:
all_medpc_scripts

- Example of what the MED-PC Script looks like that was ran when recording the behaviors

In [22]:
with open(all_medpc_scripts[0]) as f:
    lines = f.readlines()
    for line in lines[:100]:
        print(line)

- We will be using the comments in the MED-PC script(Everything after the `\` for each line) to create a name for the variables. By default, MED-PC uses a single letter as the name of the variable(programming object that holds some information).
    - This will use the medpc2excel library found in https://github.com/cyf203/medpc2excel
- Example of the variable names and their comments in the MED-PC script that we will get the descriptive names from:
    - MED-PC will squish the first few words to create the name for every variable

```
DIM P = 20000 \Port entry time stamp array

DIM Q = 2500 \US delivery time stamp array (absolute)

DIM R = 2500 \US time stamp array (relative to last CS)

DIM W = 2500 \ITI values used for CS

DIM S = 2500 \CS presentation values (absolute - every time light turns on)

DIM N = 20000 \Port exit time stamp array

DIM K = 2500 \CS type

DIM B = 2500 \shock intensity
```

- In the MED-PC recording files, there are values that are labelled with letters. These data type of these letters is described in the MED-PC script file that we just looked at.

In [23]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

## **NOTE: Please make sure that the corresponding `.mpc` file (aka the MED-PC script) that was ran to create the log file, is also in the same folder as the recording files. This notebook will fail to extract the data from the recording file if it is missing**

- Spreadsheet of the data from the MED-PC recording file for all the files combined into one.
- **NOTE: Each row does not correspond to the same data point. Each row represents the "n"-th data point for each category for each file. AKA, the first row is the first data point for the time the subject entered the port and the first data point for the time that the tone that is played, two seperate things. The second row is the second data point for all the categories, the third row is the third data point and so on. This repeats for all data points in a given file, then it starts over to the next file where the first row of that file is the first for all its data points.**

In [24]:
concatted_medpc_df = extract.dataframe.get_medpc_dataframe_from_list_of_files(medpc_files=all_med_pc_file)

In [25]:
concatted_medpc_df.head()

In [26]:
concatted_medpc_df.tail()

- Combining the recording and the metadata into one dataframe

In [27]:
recording_and_metadata_df = concatted_medpc_df.merge(metadata_df, left_on='file_path', right_on='index')

In [28]:
recording_and_metadata_df.head()

## Getting the cage numbers and the dates so that we can include it in the file name

- We will get the metadata from the recording files and use it to name the files we will create. This will help organize the files and make it easy to know where each file came from 

- Getting the group numbers(original cage names)

In [29]:
# removing blank spaces
group_numbers = ["_".join(number.split()) for number in recording_and_metadata_df["Group"].unique() if number]
# sorting numbers
group_numbers = sorted(group_numbers)
group_numbers_for_title = "_".join(group_numbers)

In [30]:
group_numbers_for_title

- Getting the cage numbers

In [31]:
# removing blank spaces
cage_numbers = ["_".join(number.split()) for number in recording_and_metadata_df["cage"].unique() if number]
# sorting numbers
cage_numbers = sorted(cage_numbers)
cage_numbers_for_title = "_".join(cage_numbers)

In [32]:
cage_numbers_for_title

- Getting the dates

In [33]:
# Getting the first and last recording date to get a range
earliest_date = recording_and_metadata_df["date"].min()
latest_date = recording_and_metadata_df["date"].max()

In [34]:
earliest_date

In [35]:
latest_date

- Getting the subject names

In [36]:
recording_and_metadata_df["Subject"].unique()

In [37]:
# removing blank spaces
subject_numbers = [number for number in recording_and_metadata_df["Subject"].unique() if number]
# sorting numbers
subject_numbers = sorted(subject_numbers)
subject_numbers_for_title = "subj_{}".format("_".join(subject_numbers))

In [38]:
subject_numbers_for_title

- Getting the experiment name

In [39]:
experiment_names = ["_".join(name.split()) for name in recording_and_metadata_df["Experiment"].unique() if name]
# sorting experiment_names
experiment_names = sorted(experiment_names)
experiment_names_for_title = "AND".join(experiment_names)

In [40]:
experiment_names_for_title

- Getting the box numbers

In [41]:
# removing blank spaces
box_numbers = ["_".join(number.split()) for number in recording_and_metadata_df["Box"].unique() if number]
# sorting numbers
box_numbers = sorted(box_numbers)
box_numbers_for_title = "_".join(box_numbers)

In [42]:
box_numbers_for_title

- Getting the script names

In [43]:
# removing blank spaces
script_names = ["_".join(name.split()) for name in recording_and_metadata_df["MSN"].unique() if name]
# sorting names
script_names = sorted(script_names)
script_names_for_title = "_".join(script_names)

In [44]:
script_names_for_title

## Saving the dataframes(spreadsheets to files) with the metadata as part of the name

- Making necessary directories
    - If you want to use any of the other metadata as part of the name, you will have to swap out the variables in the `format()` and change the name of the folder to match your new name. The variable names are the word that's in front of the `=` at the last line of each cell. The `{}` are where the metadata variables will be inserted into the file name. For more information on formatting strings: https://www.w3schools.com/python/ref_string_format.asp
    - You can also just manually rename the files by replacing everything in `""` and removing the `.format()` part
- **NOTE: You may get an error that the file does not exist. If this is the case, it could be the file name is too long(an issue that may happen when using Jupyter Notebooks on Windows)**

In [45]:
output_directory = "./proc/extracted_recording_data_and_metadata/experiment_{}_cage_{}_date_{}_{}".format(experiment_names_for_title, cage_numbers_for_title, earliest_date, latest_date)

In [46]:
output_directory

In [47]:
os.makedirs(output_directory, exist_ok=True)

In [48]:
metadata_df.to_csv(os.path.join(output_directory, "metadata_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
# metadata_df.to_excel(os.path.join(output_directory, "metadata_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

In [49]:
concatted_medpc_df.to_csv(os.path.join(output_directory, "MEDPC_recording_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
# concatted_medpc_df.to_excel(os.path.join(output_directory, "MEDPC_recording_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

In [50]:
recording_and_metadata_df.to_csv(os.path.join(output_directory, "recording_metadata_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
# recording_and_metadata_df.to_excel(os.path.join(output_directory, "recording_and_metadata_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))